In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib, os
matplotlib.use('Agg')
%matplotlib inline
from rfh_subsampling_analysis import *
import pandas as pd
import seaborn as sns
pd.set_option('precision', 4)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
            
model_directory = '../../../../rfh_importance_eval/'
class_dirs = [model_directory+'/{}/'.format(d) for d in os.listdir(model_directory) if os.path.isdir(model_directory+'/{}/'.format(d))]
k = 1

In [2]:
gather_df = gather_dir_metrics(model_directory, k, labels=['PriA-SSB AS'])
col_indices = list(range(6)) + list(range(9, 12)) + list(range(15, 63)) + list(range(87, 110))
col_indices = [i for i in col_indices if 'Mean' not in gather_df.columns[i] and 'Median' not in gather_df.columns[i]]
col_indices = [i for i in col_indices if 'PriA-SSB AS' in gather_df.columns[i]]
gather_df.columns = [col.replace('PriA-SSB AS', 'PriA-SSB prospective') for col in gather_df.columns]
metric_names = list(gather_df.columns.values[col_indices])
gather_df = gather_df[metric_names]
gather_df = gather_df.dropna(axis=0)
gather_df = gather_df.xs('test_metrics', level='set')
gather_df = gather_df.drop('Folds Mean', level='fold')
gather_df = gather_df.drop('Folds Median', level='fold')
gather_df = gather_df.reset_index(level=1, drop=True)

In [3]:
gather_df

,ROC AUC PriA-SSB prospective,BEDROC AUC PriA-SSB prospective,PR auc.integral PriA-SSB prospective,NEF_0.1 % PriA-SSB prospective,NEF_0.15 % PriA-SSB prospective,NEF_0.5 % PriA-SSB prospective,NEF_1 % PriA-SSB prospective,NEF_2 % PriA-SSB prospective,NEF_5 % PriA-SSB prospective,NEF_10 % PriA-SSB prospective,NEF_20 % PriA-SSB prospective,EF_0.1 % PriA-SSB prospective,EF_0.15 % PriA-SSB prospective,EF_0.5 % PriA-SSB prospective,EF_1 % PriA-SSB prospective,EF_2 % PriA-SSB prospective,EF_5 % PriA-SSB prospective,EF_10 % PriA-SSB prospective,EF_20 % PriA-SSB prospective,NEF AUC PriA-SSB prospective,n_hits_100 PriA-SSB prospective,n_hits_250 PriA-SSB prospective,n_hits_500 PriA-SSB prospective,n_hits_1000 PriA-SSB prospective,n_hits_2500 PriA-SSB prospective,n_hits_5000 PriA-SSB prospective,n_hits_10000 PriA-SSB prospective
model,,,,,,,,,,,,,,,,,,,,,,,,,,,
rule_0,0.8730,0.7270,0.4042,0.6364,0.5455,0.5556,0.6296,0.6852,0.7407,0.7778,0.7963,259.2593,222.2222,111.1111,62.9630,34.2593,14.8148,7.7778,3.9815,0.7491,30.0,35.0,37.0,38.0,42.0,43.0,47.0
rule_1,0.8828,0.7042,0.3307,0.6364,0.5758,0.4815,0.5370,0.6111,0.7037,0.7407,0.8148,259.2593,234.5679,96.2963,53.7037,30.5556,14.0741,7.4074,4.0741,0.7203,25.0,30.0,33.0,37.0,40.0,44.0,48.0
rule_2,0.8897,0.7285,0.3273,0.5909,0.6364,0.4630,0.5370,0.6481,0.7222,0.7963,0.8333,240.7407,259.2593,92.5926,53.7037,32.4074,14.4444,7.9630,4.1667,0.7533,25.0,31.0,36.0,39.0,44.0,45.0,48.0


---
# top250 Results

In [4]:
# load training data
file_list = ['../../../dataset/fixed_dataset/fold_5/file_{}.csv'.format(i) for i in range(5)]
train_pd = read_merged_data(file_list)
test_pd = pd.read_csv('../../../dataset/pria_prospective.csv.gz', compression='gzip')

y_train = train_pd['Keck_Pria_AS_Retest'].as_matrix()
y_test = test_pd['Keck_Pria_AS_Retest'].as_matrix()
# load training indices
train_indices_dict = {}
for model in gather_df.index:
    train_indices_file = model_directory+'/{}/fold_0/train_indices.npy'.format(model)
    train_indices_dict[model] = np.load(train_indices_file)

In [5]:
for model in gather_df.index:
    actives_count = np.where(y_train[train_indices_dict[model]] == 1)[0].shape[0]
    inactives_count = np.where(y_train[train_indices_dict[model]] == 0)[0].shape[0]
    top250_score = gather_df['n_hits_250 PriA-SSB prospective'][model]
    print('{}: actives count: {}\t inactives count: {}\t top250: {}'.format(model, actives_count, inactives_count, top250_score))

rule_0: actives count: 19	 inactives count: 1372	 top250: 35.0
rule_1: actives count: 23	 inactives count: 20	 top250: 30.0
rule_2: actives count: 29	 inactives count: 28	 top250: 31.0


In [6]:
s0 = set(np.where(y_train[train_indices_dict['rule_0']] == 0)[0])
s1 = set(np.where(y_train[train_indices_dict['rule_1']] == 0)[0])
s2 = set(np.where(y_train[train_indices_dict['rule_2']] == 0)[0])

assert s0.intersection(s1) == s1
assert s0.intersection(s2) == s2

In [7]:
s0 = set(np.where(y_train[train_indices_dict['rule_0']] == 1)[0])
s1 = set(np.where(y_train[train_indices_dict['rule_1']] == 1)[0])
s2 = set(np.where(y_train[train_indices_dict['rule_2']] == 1)[0])

assert s0.intersection(s1) == s0
assert s0.intersection(s2) == s0

From the above assertions, we see that all the actives in rule_0 exist in rule_1 and rule_2. So adding extra actives in rule_1 and rule_2 did not imporve top250. However, rule_0 has all the inactives of rule_1 and rule_2 but the extra inactives helped improve top250 by 4 points. Would be helpful if we scrutinize these ~1300 extra inactives to determine why they improve the results.